# Dashboard for Symulating Cyclic Lockdown Scenarios

In [ ]:
from scipy import interpolate
import dash_table
import mydcc
import dash_bootstrap_components as dbc
from datetime import datetime as dt
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
import numpy as np
pd.options.mode.chained_assignment = None  
from scipy.integrate import odeint


############################################ the model ################################################

gamma_M = 1/7
gamma_H = 1/7
gamma_I = 1/7
t_EM = 5.1
t_MH = 6.9
t_HI = 4.5
t_IF = 6.1

beta_H = 0.15
beta_I = 0.15


days = 256

def seirmodel(y, t, beta_M, beta_ML, beta_H, beta_I, gamma_M, gamma_H, gamma_I, t_EM, t_MH, t_HI, t_IF, p_H, p_I, p_F, L, N=9_304_016):
    S, E, M, H, I, R, F = y
    
    m = L[0]+L[1]
    good = list(range(1, m+1))
    def beta_R0M(t, L):
        if (t < 88):
            return beta_M
        elif (t<155):
            return beta_ML
        elif (int(t)%m > L[0] or int(t)%m == 0):
            return beta_ML
        else:
            return beta_M
    
    dSdt = -(beta_R0M(t, L) * M + beta_H * H + beta_I * I)* S / N
    dEdt = (beta_R0M(t, L) * M + beta_H * H + beta_I * I)* S / N - E / t_EM
    dMdt = E / t_EM - gamma_M * M - p_H * M / t_MH
    dHdt = p_H * M / t_MH - gamma_H * H - p_I * H / t_HI
    dIdt = p_I * H / t_HI - gamma_I * I - p_F * I / t_IF
    
    dRdt = gamma_M * M + gamma_I * I + gamma_H * H
    dFdt = p_F * I / t_IF
    return dSdt, dEdt, dMdt, dHdt, dIdt, dRdt, dFdt 




def Model(days, initial_cases, initial_date, L, R, beta_M, beta_ML, p_H, p_I, p_F):
    S0, E0, M0, H0, I0, R0, F0 =  9_304_016-initial_cases, initial_cases, 0.0, 0.0, 0.0, 0.0, 0.0
    
    
    t = np.linspace(0, days-1,days) 
    y0 = S0, E0, M0, H0, I0, R0, F0 

   
    ret = odeint(seirmodel, y0, t, args=(beta_M, beta_ML, beta_H, beta_I, gamma_M, gamma_H, gamma_I, t_EM, t_MH, t_HI, t_IF, p_H, p_I, p_F, L, 9_304_016))
    
    
    S, E, M, H, I, R, F = ret.T
    
    
    dates = pd.date_range(start=np.datetime64(initial_date), periods=days, freq="D")
    return dates, S, E, M, H, I, R, F



############################################ the dash app layout ################################################

external_stylesheets = [dbc.themes.BOOTSTRAP]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Tabular Rasa"
#at the beginning, my Dashboard did not show anything, hence the app name 'Tabular Rasa' :)

controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label('Date of first infection'),
                html.Br(),
                dcc.DatePickerSingle(
                    day_size=39,  
                    display_format="DD.MM.YYYY",
                    date='2020-01-07',
                    id='initial_date',
                    min_date_allowed=dt(2019, 12, 1),
                    max_date_allowed=dt(2020, 6, 6),
                    initial_visible_month=dt(2020, 1, 15),
                    placeholder="test"
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Initial Cases"),
                dbc.Input(
                    id="initial_cases", type="number", placeholder="initial_cases",
                    min=1, max=9_304_016, step=1, value=1,
                )
            ]
        ),

        
    
        dbc.FormGroup(
            [
                dbc.Label('Probability of requiring Hospital Care when infected (%)'),
                html.Br(),
                dcc.Slider(
                    id='p_H',
                    min=0.01,
                    max=100.0,
                    step=0.01,
                    value=10.0,
                    tooltip={'always_visible': True, "placement": "bottom"}
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label('Probability of requiring Intensive care if requiring Hospital Care (%)'),
                html.Br(),
                dcc.Slider(
                    id='p_I',
                    min=0.01,
                    max=100.0,
                    step=0.01,
                    value=15.0,
                    tooltip={'always_visible': True, "placement": "bottom"}
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label('Probability of dying if requiring Intensive Care (%)'),
                dcc.Slider(
                    id='p_F',
                    min=0.01,
                    max=100.0,
                    step=0.01,
                    value=50.0,
                    tooltip={'always_visible': True, "placement": "bottom"}
                ),
            ]
        ),
        
        dbc.FormGroup(
            [
                dbc.Label('Cyclic Lockdown strategies'),
                dash_table.DataTable(
                    id='L_table',
                    columns=[
                        {"name": "Measure", "id": "Measure"},
                        {"name": "Number of days", "id": "Number of days",
                         "editable": True, "type": "numeric"},
                    ],
                    data=[
                        {
                            "Measure": i[0],
                            "Number of days": i[1],
                        }
                        for i in [("No lockdown", 4), ("Lockdown", 10)]
                    ],
                    style_cell_conditional=[
                        {'if': {'column_id': 'Measure'},
                         'width': '5px'},
                        {'if': {'column_id': 'Number of days'},
                         'width': '10px'},
                    ],
                    style_cell={'textAlign': 'left',
                                'fontSize': 16, 'font-family': 'Helvetica'},
                    style_header={
                        'backgroundColor': 'pink',
                        'fontWeight': 'bold'
                    },

                ),
            ]
        ),
        
        dbc.FormGroup(
            [
                dbc.Label('No Lockdown and Lockdown infection rates'),
                dash_table.DataTable(
                    id='R_table',
                    columns=[
                        {"name": "Measure", "id": "Measure"},
                        {"name": "R_0 values", "id": "R_0 values",
                         "editable": True, "type": "numeric"},
                    ],
                    data=[
                        {
                            "Measure": i[0],
                            "R_0 values": i[1],
                        }
                        for i in [("No lockdown", 3.85), ("Lockdown", 0.25)]
                    ],
                    style_cell_conditional=[
                        {'if': {'column_id': 'Measure'},
                         'width': '5px'},
                        {'if': {'column_id': 'R_0 values'},
                         'width': '10px'},
                    ],
                    style_cell={'textAlign': 'left',
                                'fontSize': 16, 'font-family': 'Helvetica'},
                    style_header={
                        'backgroundColor': 'pink',
                        'fontWeight': 'bold'
                    },

                ),
            ]
        ),
        
        dbc.Button("Apply", id="submit-button-state",
                   color="primary", block=True)
    ],
    body=True,
)

# layout for the whole page
app.layout = dbc.Container(
    [
        
        dbc.Jumbotron(
            [
                dbc.Container(
                    [
                        html.H1("COVID-19 Dashboard", className="display-3"),
                        html.P(
                            "Interactively simulate different Cyclic Lockdown scenarios. ",
                            className="lead",
                        ),
                        html.Hr(className="my-2"),
                        dcc.Markdown('''

                            Special thanks for Amir for helping me spot and fix the very last error. :)
                            '''
                                     )
                    ],
                    fluid=True,
                )
            ],
            fluid=True,
            className="jumbotron bg-white text-dark"
        ),
        
        dbc.Row(
            [
                
                dbc.Col(controls, md=3),
                
                dbc.Col(
                    [
                        
                        
                        
                        dcc.Graph(id='graph2'),
                        
                        dcc.Graph(id='graph3')
                        
                        
                        
                    ],
                    md=9
                ),
            ],
            align="top",
        ),
    ],
   
    fluid=True,
)



############################################ the dash app callbacks ################################################


@app.callback(
    [
     dash.dependencies.Output('graph2', 'figure'),
     dash.dependencies.Output('graph3', 'figure')
    ],
    
    [dash.dependencies.Input('submit-button-state', 'n_clicks')],

    [
     dash.dependencies.State('initial_date', 'date'),
     dash.dependencies.State('initial_cases', 'value'),
     dash.dependencies.State('p_H', 'value'),
     dash.dependencies.State('p_I', 'value'),
     dash.dependencies.State('p_F', 'value'),
     dash.dependencies.State('L_table', 'data'),
     dash.dependencies.State('L_table', 'columns'),
     dash.dependencies.State('R_table', 'data'),
     dash.dependencies.State('R_table', 'columns')
     ]
)


def update_graph1(_, initial_date, initial_cases,  p_H,  p_I, p_F, L_data, L_columns, R_data, R_columns):
    
    
    last_initial_date, last_initial_cases, last_p_H, last_p_I, last_p_F,  = "2020-01-07", 1,  10.0, 10.0, 30.0, 
    if not (initial_date and initial_cases and p_H and p_I and p_F ):
        initial_date, initial_cases, p_H, p_I, p_F = last_initial_date, last_initial_cases, last_p_H, last_p_I, last_p_F

    L = [datapoint["Number of days"] for datapoint in L_data]
    R = [datapoint["R_0 values"] for datapoint in R_data]
    beta_M = R[0] * gamma_M
    beta_ML = R[1] * gamma_M
    
    dates, S, E, M, H, I, R, F = Model(days, initial_cases, initial_date, L, R, beta_M, beta_ML, float(p_H)/100, float(p_I)/100, float(p_F)/100 )
    dates1 =  dates = pd.date_range(start=np.datetime64('2020-06-09'), periods=100, freq="D")                                 
    S1 = []
    E1 = []
    M1 = []
    H1 = []
    I1 = []
    R1 = []
    F1 = []
    
    for i in range(84):
        S1.append(S[154+i])
        E1.append(E[154+i])
        M1.append(M[154+i])
        H1.append(H[154+i])
        I1.append(I[154+i])
        R1.append(R[154+i])
        F1.append(F[154+i])
    S1=np.array(S1)
    E1=np.array(E1)
    M1=np.array(M1)
    H1=np.array(H1)
    I1=np.array(I1)
    R1=np.array(R1)
    F1=np.array(F1)

    
        
        
    return  {
        'data': [
            {'x': dates, 'y': E1.astype(int), 'type': 'line', 'name': 'Exposed'},
            {'x': dates, 'y': M1.astype(int), 'type': 'line', 'name': 'Mild infection'},
            {'x': dates, 'y': H1.astype(int), 'type': 'line', 'name': 'Hospital care'},
            {'x': dates, 'y': I1.astype(int), 'type': 'line', 'name': 'Intensive care'},
            {'x': dates, 'y': F1.astype(int), 'type': 'line', 'name': 'Dead'}
            ],
        
        'layout': {
            'title': 'Infected compartments over time'
        }
    }, {
        'data': [
            
            {'x': dates, 'y': H1.astype(int), 'type': 'line', 'name': 'Hospital care'},
            {'x': dates, 'y': I1.astype(int), 'type': 'line', 'name': 'Intensive care'}
            
            ],
        
        'layout': {
            'title': 'Compartments requiring Hospitalization or Intensive Care'
        }
    }



if __name__ == '__main__':
    app.run_server(debug = False, port = 3004)
 


